In [1]:
# !pip install transformers
# !pip install joblib
import numpy as np
import pydicom
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as ptchs
import cv2

from tokenizers import Tokenizer
from transformers import ConvNextFeatureExtractor, TFConvNextModel, ConvNextConfig, ConvNextModel

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

from tensorflow.keras import Model, Sequential 
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import Conv2D, MaxPool2D, AveragePooling2D, GlobalAveragePooling1D, GlobalAveragePooling2D, MaxPooling2D , Conv3D, Layer, MaxPooling2D, Dropout, Flatten, Dense, GRU, ConvLSTM2D, Input, BatchNormalization, TimeDistributed, MaxPooling3D, Bidirectional, LSTM
from tensorflow.keras.optimizers import Adam
from random import shuffle
from tensorflow.keras.regularizers import l2
from keras.models import Model, load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.constraints import max_norm

print(tf.__version__)
print(keras.__version__)
import re
import math
import random
# import bisect
np.random.seed(8)
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
import warnings
warnings.filterwarnings(action='once')

MAX_SEQ_LENGTH = 200
NUM_FEATURES = 1024
IMG_SIZE = 512
BATCH_SIZE = 4
EPOCHS = 2
NUM_SCANS = 8
NUM_CHANNELS = 1
INPUT_DIM = 256

/home/jupyter-paige/.conda/envs/tf-gpu/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.8.1
2.8.0
Num GPUs Available:  1


In [2]:
CROPPED = 0
MASKED = 1
ORIGINAL = 2
IMAGE_PATH = '/home/shared/nps/imgs/'
BATCH_SIZE = 4

In [3]:
all_ids = pd.read_csv('all_ids_updated.csv')
all_ids = all_ids[all_ids.contains_lung == True].sample(frac=1).reset_index(drop=True)
all_ids.ycoord = all_ids.ycoord.replace('True', '1.0').astype('float')

/tmp/ipykernel_4138698/1656503385.py:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  all_ids = pd.read_csv('all_ids_updated.csv')


In [4]:
train_df = pd.read_csv('train_df_upd.csv').drop(columns='Unnamed: 0')
test_df = pd.read_csv('test_df_upd.csv').drop(columns='Unnamed: 0')
val_df = pd.read_csv('val_df_upd.csv').drop(columns='Unnamed: 0')

In [5]:
train_df = pd.concat([train_df, val_df])
train_all_ids = all_ids[all_ids.StudyInstanceUID.isin(train_df.StudyInstanceUID)]
study_ids = train_all_ids.pivot_table(index='StudyInstanceUID').sort_values('pe_present_on_image').index

fold_0_ids = [study_ids[i] for i in range(len(study_ids)) if i%5 == 0]
fold_1_ids = [study_ids[i] for i in range(len(study_ids)) if i%5 == 1]
fold_2_ids = [study_ids[i] for i in range(len(study_ids)) if i%5 == 2]
fold_3_ids = [study_ids[i] for i in range(len(study_ids)) if i%5 == 3]
fold_4_ids = [study_ids[i] for i in range(len(study_ids)) if i%5 == 4]

In [6]:
all_ids.ycoord = all_ids.ycoord.astype('float')

SLAB_SIZE = 3

all_ids_slab = all_ids.sort_values(['StudyInstanceUID', 'ycoord'])
all_ids_slab['endStudyUID'] = all_ids_slab.StudyInstanceUID.shift(SLAB_SIZE - 1)

all_ids_slab['pe_in_slab'] = all_ids_slab.pe_present_on_image
for i in range(1, SLAB_SIZE):
    all_ids_slab['pe_' + str(i)] = all_ids_slab.pe_present_on_image.shift(i)
    all_ids_slab['SOP_' + str(i)] = all_ids_slab.SOPInstanceUID.shift(i)
    all_ids_slab['pe_in_slab'] = all_ids_slab['pe_in_slab'] + all_ids_slab['pe_' + str(i)]

all_ids_slab['pe_in_slab'] = all_ids_slab['pe_1']

all_ids_slab = all_ids_slab[all_ids_slab.StudyInstanceUID == all_ids_slab.endStudyUID]

all_ids_slab = all_ids_slab.reset_index(drop=True)
print(len(all_ids_slab[all_ids_slab.pe_in_slab ==True]), len(all_ids_slab[all_ids_slab.pe_present_on_image == True]))

train_slab = all_ids_slab[all_ids_slab.StudyInstanceUID.isin(train_df.StudyInstanceUID)]
test_slab = all_ids_slab[all_ids_slab.StudyInstanceUID.isin(test_df.StudyInstanceUID)]
print(len(train_slab[train_slab.pe_in_slab ==True]), len(test_slab[test_slab.pe_in_slab == True]))

95746 95744
86853 8893


In [7]:
train_slab['4fold'] = train_slab.StudyInstanceUID.isin(fold_0_ids)
train_slab['3fold'] = train_slab.StudyInstanceUID.isin(fold_1_ids)
train_slab['2fold'] = train_slab.StudyInstanceUID.isin(fold_2_ids)
train_slab['1fold'] = train_slab.StudyInstanceUID.isin(fold_3_ids)
train_slab['0fold'] = train_slab.StudyInstanceUID.isin(fold_4_ids)

/tmp/ipykernel_4138698/108160448.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_slab['4fold'] = train_slab.StudyInstanceUID.isin(fold_0_ids)
/tmp/ipykernel_4138698/108160448.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_slab['3fold'] = train_slab.StudyInstanceUID.isin(fold_1_ids)
/tmp/ipykernel_4138698/108160448.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [8]:
print(train_slab[train_slab['4fold'] == True].pe_in_slab.sum())
print(train_slab[train_slab['3fold'] == True].pe_in_slab.sum())
print(train_slab[train_slab['2fold'] == True].pe_in_slab.sum())
print(train_slab[train_slab['1fold'] == True].pe_in_slab.sum())
print(train_slab[train_slab['0fold'] == True].pe_in_slab.sum())

17096.0
17544.0
17197.0
17418.0
17598.0


In [9]:

print(f"Total videos for training: {len(train_slab)}")
print(f"Total videos for testing: {len(test_slab)}")


class DataSlabGenerator(Sequence):
    """Generates data for Keras
    Sequence based data generator. Suitable for building data generator for training and prediction.
    """
    def __init__(self, list_IDs, labels, image_path, img_type,
                 n_channels=1, to_fit=True, batch_size=32, dim=256, num_pos=None,
                 n_classes=2, shuffle=True, verbose=False, set_type='valid', 
                 convex_hull = False, full_set = None, resample = False):
        """Initialization
        :param list_IDs: list of all 'label' ids to use in the generator
        :param labels: list of image labels (file names)
        :param image_path: path to images location
        :param mask_path: path to masks location
        :param to_fit: True to return X and y, False to return X only
        :param batch_size: batch size at each iteration
        :param dim: tuple indicating image dimension
        :param n_channels: number of image channels
        :param n_classes: number of output masks
        :param shuffle: True to shuffle label indexes after every epoch
        """
        self.list_IDs = list_IDs
        self.labels = labels
        self.image_path = image_path
        self.to_fit = to_fit
        self.batch_size = batch_size
        self.dim = dim
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.path_dicom = image_path
        self.verbose = verbose
        self.img_type = img_type
        warnings.filterwarnings(action='ignore')
        self.full_set = full_set
        self.resample = resample
        self.epoch = 0
        self.set_type = set_type
        self.num_pos = num_pos
        self.on_epoch_end()

        
    def __len__(self):
        """Denotes the number of batches per epoch
        :return: number of batches per epoch
        """
        return int(np.floor(len(self.list_IDs) / self.batch_size))


    def __getitem__(self, index):
        """Generate one batch of data
        :param index: index of the batch
        :return: X and y when fitting. X only when predicting
        """
        
        indexes = self.list_IDs[index * self.batch_size:((index+1) * self.batch_size)]
        X = np.zeros([self.batch_size,self.dim,self.dim,3])
        y = np.zeros([self.batch_size, 1])
        for i in range(0,self.batch_size):
            X[i], y[i] = self._load_dicom(indexes[i])
            
        if self.verbose == True:
            fig, ax = plt.subplots(self.batch_size, 1, figsize=[12, 12*(self.batch_size/2)])
            for i in range(self.batch_size):
                ax[i].imshow(X[i])
                ax[i].axis('off')
        
        X = X/125.5 -1
            
        if self.to_fit:
            return (X, y)
        else:
            return (X)
        
    def on_epoch_end(self):
        """Updates indexes after each epoch
        """
        if self.resample == True:
            print('resampling')
            if self.set_type == 'valid':
                sample_set = self.full_set[self.full_set[str(self.epoch%5)+'fold'] == True]
            if self.set_type == 'train':
                sample_set = self.full_set[self.full_set[str(self.epoch%5)+'fold'] == False]
                print(str(self.epoch%5)+'fold')
            sample_set = sample_set.reset_index(drop=True)
            set_pos = sample_set[sample_set.pe_in_slab ==True]
            set_neg = sample_set[sample_set.pe_in_slab == False].sample(n=len(set_pos))
            self.labels = pd.concat([set_pos,set_neg]).sample(frac=1).reset_index(drop=True)

            self.epoch = self.epoch + 1
            
        self.list_IDs = np.arange(len(self.labels))
        if self.shuffle == True:
            np.random.shuffle(self.list_IDs)

    def _load_dicom(self, index):
        slice = self.labels.iloc[index] 

        frames = np.zeros((self.dim,self.dim,3))
        frame = cv2.imread(self.image_path+slice.StudyInstanceUID + '_'+ slice.SeriesInstanceUID
                      + '_' + slice.SOPInstanceUID +'.png')
        frame = frame[:,:,self.img_type]
        frames[:,:,0] = frame

        frame = cv2.imread(self.image_path+slice.StudyInstanceUID + '_'+ slice.SeriesInstanceUID
                      + '_' + slice.SOP_1 +'.png')
        frame = frame[:,:,self.img_type]
        frames[:,:,1] = frame    

        frame = cv2.imread(self.image_path+slice.StudyInstanceUID + '_'+ slice.SeriesInstanceUID
                      + '_' + slice.SOP_2 +'.png')
        frame = frame[:,:,self.img_type]
        frames[:,:,2] = frame   

        if self.verbose == True:
            print(np.shape(frame))
            plt.imshow(frame)
            plt.show()

        frames = frames[None, ...]
        y = np.array([int(slice.pe_in_slab)])
        y = y[None, ...]

        return frames, y
        
BATCH_SIZE = 64
IMG_TYPE = ORIGINAL

    
training_slab_generator = DataSlabGenerator([],
                                   train_slab, 
                                   IMAGE_PATH, 
                                   img_type = IMG_TYPE,
                                   verbose=False, 
                                   resample=True, 
                                   full_set=train_slab,
                                   n_channels=3, 
                                   set_type='train',
                                   batch_size=BATCH_SIZE)
validation_slab_generator = DataSlabGenerator([],
                                   train_slab, 
                                   IMAGE_PATH, 
                                   img_type = IMG_TYPE,
                                   verbose=False, 
                                   resample=True,
                                   set_type = 'valid',
                                   full_set = train_slab,
                                   n_channels=3, 
                                   batch_size=BATCH_SIZE)
test_slab_generator = DataSlabGenerator(np.arange(0, len(test_slab)),
                                   test_slab, 
                                   IMAGE_PATH, 
                                   img_type = IMG_TYPE,
                                   verbose=False, 
                                   resample=False, 
                                   n_channels=3, 
                                   batch_size=BATCH_SIZE)


Total videos for training: 1047595
Total videos for testing: 113175
resampling
0fold
resampling


In [10]:
from keras_cv_attention_models import resnest
res = resnest.ResNest50(pretrained='imagenet', num_classes=1, input_shape=(448,448,3), classifier_activation='sigmoid')

2022-10-03 12:22:27.997681: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-03 12:22:29.552496: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11194 MB memory:  -> device: 0, name: NVIDIA GeForce GTX TITAN X, pci bus id: 0000:06:00.0, compute capability: 5.2


>>>> Load pretrained from: /home/jupyter-paige/.keras/models/resnest50_imagenet.h5


In [15]:
data_augmentation = keras.Sequential(
                    [
                        layers.Resizing(512,512),
                        layers.RandomCrop(448, 448),
                        layers.Resizing(448,448)
                    ],
                    name="data_augmentation",)
inputs = layers.Input(shape=(256,256,3))
augmented = data_augmentation(inputs)
outputs = res(augmented)
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()


model.compile(
    optimizer=keras.optimizers.schedules(learning_rate=0.001, beta_1=0.9, beta_2=0.999, decay=0.01),
    loss=tf.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

filename = 'i-pan'
checkpoint_filepath = '/home/shared/model_checkpoint_paige/singlescan-3channel/' + filename
checkpoint = keras.callbacks.ModelCheckpoint(
    checkpoint_filepath, save_weights_only=True, save_best_only=True, verbose=1
)

history = model.fit(
    training_slab_generator,
    validation_data = validation_slab_generator,
    epochs = 5,
    callbacks=[checkpoint,
              ],
    shuffle=True
)



Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, 448, 448, 3)      0         
 al)                                                             
                                                                 
 resnest50 (Functional)      (None, 1)                 25500865  
                                                                 
Total params: 25,500,865
Trainable params: 25,436,289
Non-trainable params: 64,576
_________________________________________________________________


ValueError: Could not interpret optimizer identifier: <keras.optimizer_v2.learning_rate_schedule.CosineDecay object at 0x7f1e70e81000>

In [ ]:
data_augmentation = keras.Sequential(
                    [
                        layers.RandomRotation(factor=0.1, fill_mode='constant'),
                        layers.RandomCrop(240, 240),
                        layers.Resizing(256,256)
                    ],
                    name="data_augmentation",)
inputs = layers.Input(shape=(256,256,3))
augmented = data_augmentation(inputs)
res = keras.applications.resnet_v2.ResNet50V2(
                        include_top=False,
                        weights='imagenet',
#                         input_tensor=augmented,
                        pooling='avg',)
x = res(augmented)
x = layers.Dense(512)(x)
x = layers.Dropout(0.8)(x)    
outputs = Dense(1, activation = 'sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, decay=0.01),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

filename = 'resnet-2'
checkpoint_filepath = '/home/shared/model_checkpoint_paige/singlescan-3channel/' + filename
checkpoint = keras.callbacks.ModelCheckpoint(
    checkpoint_filepath, save_weights_only=True, save_best_only=True, verbose=1
)

history = model.fit(
    training_slab_generator,
    validation_data = validation_slab_generator,
    epochs = 10,
    callbacks=[checkpoint,
              ],
    shuffle=True,
#     steps_per_epoch=15935
)



Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 256, 256, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, 256, 256, 3)      0         
 al)                                                             
                                                                 
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
 dense_18 (Dense)            (None, 512)               1049088   
                                                                 
 dropout_9 (Dropout)         (None, 512)               0         
                                                                 
 dense_19 (Dense)            (None, 1)                 513       
                                                          

In [39]:
data_augmentation = keras.Sequential(
                    [
                        layers.RandomRotation(factor=0.2, fill_mode='constant'),
                        layers.RandomCrop(240, 240),
                        layers.Resizing(256,256)
                    ],
                    name="data_augmentation",)
inputs = layers.Input(shape=(256,256,3))
augmented = data_augmentation(inputs)
res = keras.applications.resnet_v2.ResNet50V2(
                        include_top=False,
                        weights='imagenet',
#                         input_tensor=augmented,
                        pooling='avg',)
x = res(augmented)
x = layers.Dense(512)(x)
x = layers.Dropout(0.8)(x)    
outputs = Dense(1, activation = 'sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, decay=0.01),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

filename = 'resnet-2-2'
checkpoint_filepath = '/home/shared/model_checkpoint_paige/singlescan-3channel/' + filename
checkpoint = keras.callbacks.ModelCheckpoint(
    checkpoint_filepath, save_weights_only=True, save_best_only=True, verbose=1
)

history = model.fit(
    training_slab_generator,
    validation_data = validation_slab_generator,
    epochs = 10,
    callbacks=[checkpoint,
              ],
    shuffle=True,
#     steps_per_epoch=15935
)



Model: "model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_39 (InputLayer)       [(None, 256, 256, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, 256, 256, 3)      0         
 al)                                                             
                                                                 
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
 dense_31 (Dense)            (None, 512)               1049088   
                                                                 
 dropout_17 (Dropout)        (None, 512)               0         
                                                                 
 dense_32 (Dense)            (None, 1)                 513       
                                                          

KeyboardInterrupt: 

In [ ]:
from keras_cv_attention_models import davit
davit_t = davit.DaViT_T(pretrained='imagenet', num_classes=1, classifier_activation='sigmoid', dropout=0.8, drop_connect_rate=0.2, input_shape=(256,256,3))

data_augmentation = keras.Sequential(
                    [
                        layers.RandomRotation(factor=0.2, fill_mode='constant'),
                        layers.RandomCrop(240, 240),
                        layers.Resizing(256,256)
                    ],
                    name="data_augmentation",)
inputs = layers.Input(shape=(256,256,3))
augmented = data_augmentation(inputs)
outputs = davit_t(augmented)
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, decay=0.01),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

filename = 'davit_t'
checkpoint_filepath = '/home/shared/model_checkpoint_paige/singlescan-3channel/' + filename
checkpoint = keras.callbacks.ModelCheckpoint(
    checkpoint_filepath, save_weights_only=True, save_best_only=True, verbose=1
)

history = model.fit(
    training_slab_generator,
    validation_data = validation_slab_generator,
    epochs = 5,
    callbacks=[checkpoint,
              ],
    shuffle=True,
#     steps_per_epoch=15935
)



>>>> Load pretrained from: /home/jupyter-paige/.keras/models/davit_t_imagenet.h5
Model: "model_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_50 (InputLayer)       [(None, 256, 256, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, 256, 256, 3)      0         
 al)                                                             
                                                                 
 davit_t (Functional)        (None, 1)                 27591937  
                                                                 
Total params: 27,591,937
Trainable params: 27,591,937
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
 1955/17308 [==>...........................] - ETA: 3:16:56 - loss: 0.5933 - accuracy: 0.6721

In [ ]:
davit_s = davit.DaViT_S(pretrained='imagenet', num_classes=1, classifier_activation='sigmoid', dropout=0.8, drop_connect_rate=0.2, input_shape=(256,256,3))

data_augmentation = keras.Sequential(
                    [
                        layers.RandomRotation(factor=0.2, fill_mode='constant'),
                        layers.RandomCrop(240, 240),
                        layers.Resizing(256,256)
                    ],
                    name="data_augmentation",)
inputs = layers.Input(shape=(256,256,3))
augmented = data_augmentation(inputs)
outputs = davit_s(augmented)
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, decay=0.01),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

filename = 'davit_s'
checkpoint_filepath = '/home/shared/model_checkpoint_paige/singlescan-3channel/' + filename
checkpoint = keras.callbacks.ModelCheckpoint(
    checkpoint_filepath, save_weights_only=True, save_best_only=True, verbose=1
)

history = model.fit(
    training_slab_generator,
    validation_data = validation_slab_generator,
    epochs = 5,
    callbacks=[checkpoint,
              ],
    shuffle=True,
)

